### how to you get your own data into PyTorch? 
one of the ways to do so is via *custum datasets*

In [2]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
torch.__version__

'2.7.1'

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'